In [1]:
import numpy as np
n_features = 20
term_pred = 'long_term'
data_file_name = 'data_after_y10_origin_at_10_top_20_var'
endpt = 16
eval_times = 365.25*np.r_[np.arange(2, endpt+1, 2)]
training_id_file_name = 'all_training_ID_outerloop_cohort_10_26'
l2_final=0.1
work_dir = '/home/idies/workspace/Storage/hnguye78/persistent/CARDIA_project/cvd_outcome_rerun2'


In [2]:
# %pip install tensorflow
# %pip install lifelines

#!y | pip uninstall statsmodels 
# %pip install statsmodels==0.11.0

# %pip install keras

In [3]:
from __future__ import print_function
from tensorflow.keras import backend as K

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout, Activation, LSTM, GRU, Embedding, Concatenate, Conv1D, GlobalMaxPooling1D, MaxPooling1D, GlobalAveragePooling1D, BatchNormalization, TimeDistributed
from tensorflow.keras import optimizers, layers, regularizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
import math
from lifelines import KaplanMeierFitter
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index
from sklearn.preprocessing import StandardScaler
from scipy import stats
import time


In [4]:
# snippets_dir = work_dir+ '/code/snippet'
snippets_dir = '/home/idies/workspace/Storage/hnguye78/persistent/CARDIA_project/cvd_outcome_rerun'+ '/code/snippet'

import os, sys
sys.path.append(os.path.abspath(snippets_dir))
import nnet_survival
from cox import *


In [5]:
load_dir = work_dir+ '/csv_files'

# load data:
data_full = pd.read_csv(load_dir+'/'+data_file_name+'.csv')
data_full = data_full.select_dtypes(include =[np.number])



In [6]:
data_full

,ID,time,event,C09SMKYR,C02AVGDI,C02AVGSY,CL1LDL,C08DIBAG_YEAR,CL1VLDL,C20WST1,...,DPINTAE,CL1NTRIG,C02ARMCI,C08CHOAG_YEAR,C20WGT,C08HBPAG_YEAR,C20HIPS1,C08KYSAG_YEAR,C20CHES1,DPVAE
0,100016012504,5258.5,0,12,72.0,114.0,119.000000,0,29.0,90.5,...,0.320000,147.0,33.0,0,180.0,1,105.5,0,95.5,0.450000
1,100023004268,5560.5,0,8,64.0,94.0,105.000000,0,18.0,79.0,...,0.310000,89.0,28.0,0,169.5,0,100.0,0,94.0,0.600000
2,100033323702,5381.5,0,0,73.0,105.0,160.000000,0,22.0,78.5,...,0.320000,110.0,30.0,0,169.0,0,101.5,0,93.5,0.490000
3,100056526386,5511.5,0,0,76.0,118.0,137.000000,0,18.0,96.0,...,0.380000,91.0,37.0,0,219.5,0,106.0,0,112.5,0.590000
4,100061300991,5403.5,0,11,72.0,112.0,108.000000,0,52.0,91.0,...,0.390000,260.0,33.0,0,216.5,0,103.0,0,102.0,0.560000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4941,416761219907,5439.5,0,0,63.0,98.0,150.000000,0,17.0,67.0,...,0.200000,84.0,25.0,5,120.0,0,98.5,0,82.0,0.370000
4942,416771521620,5586.5,0,0,82.0,111.0,120.000000,0,8.0,80.0,...,0.450000,38.0,36.0,0,184.0,0,106.0,0,102.0,0.830000
4943,416783315386,5563.5,0,3,64.0,109.0,90.000000,0,9.0,70.0,...,0.408445,47.0,25.0,0,126.0,0,96.5,0,87.5,0.597184
4944,416796224310,5452.5,0,12,69.0,108.0,100.000000,0,8.0,102.5,...,0.530000,40.0,39.0,0,229.0,0,122.5,0,111.0,0.790000


In [7]:
# load training id:
loaddir = work_dir+ '/csv_files'
trainingid_all = pd.read_csv(loaddir+'/'+training_id_file_name+'.csv')

## standardize feature space, then merge back to label space:
feature_space = data_full.drop(['ID','event','time'], axis = 1)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(feature_space)
scaled_feature_space = scaler.transform(feature_space)
scaled_feature_space_df = pd.DataFrame(data=scaled_feature_space[0:,0:])
scaled_feature_space_df.insert(0, 'ID', data_full['ID'], True)
label = data_full.loc[:,['ID','event','time']]
data_full = pd.merge(label, scaled_feature_space_df)

In [8]:
data_full

,ID,event,time,0,1,2,3,4,5,6,...,10,11,12,13,14,15,16,17,18,19
0,100016012504,0,5258.5,1.503322,0.300456,0.588403,0.354056,-0.081168,1.005284,0.713121,...,-0.608365,1.035478,0.366059,-0.182795,0.322823,0.231206,0.194651,-0.082092,-0.016469,-1.017178
1,100023004268,0,5560.5,0.798509,-0.565630,-1.303633,-0.127119,-0.081168,0.151872,-0.262465,...,-0.685683,0.136070,-0.727322,-0.182795,0.042367,-0.198859,-0.317435,-0.082092,-0.163959,0.058423
2,100033323702,0,5381.5,-0.611119,0.408717,-0.263013,1.763210,-0.081168,0.462204,-0.304882,...,-0.608365,0.461718,-0.289970,-0.182795,0.029012,-0.198859,-0.177775,-0.082092,-0.213123,-0.730351
3,100056526386,0,5511.5,-0.611119,0.733499,0.966811,0.972709,-0.081168,0.151872,1.179706,...,-0.144455,0.167085,1.240764,-0.182795,1.377868,-0.198859,0.241204,-0.082092,1.655084,-0.013284
4,100061300991,0,5403.5,1.327119,0.300456,0.399200,-0.024010,-0.081168,2.789693,0.755538,...,-0.067137,2.787773,0.366059,-0.182795,1.297738,-0.198859,-0.038116,-0.082092,0.622654,-0.228404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4941,416761219907,0,5439.5,-0.611119,-0.673891,-0.925226,1.419514,-0.081168,0.074289,-1.280468,...,-1.536185,0.058535,-1.383351,2.348288,-1.279778,-0.198859,-0.457095,-0.082092,-1.343879,-1.590832
4942,416771521620,0,5586.5,-0.611119,1.383063,0.304598,0.388426,-0.081168,-0.623958,-0.177631,...,0.396774,-0.654788,1.022088,-0.182795,0.429663,-0.198859,0.241204,-0.082092,0.622654,1.707678
4943,416783315386,0,5563.5,-0.082509,-0.565630,0.115394,-0.642663,-0.081168,-0.546375,-1.025967,...,0.075478,-0.515225,-1.383351,-0.182795,-1.119518,-0.198859,-0.643308,-0.082092,-0.803083,0.038228
4944,416796224310,0,5452.5,1.503322,-0.024326,0.020793,-0.298967,-0.081168,-0.623958,1.731124,...,1.015321,-0.623774,1.678117,-0.182795,1.631613,-0.198859,1.777461,-0.082092,1.507594,1.420851


In [9]:
data_full = data_full

In [10]:
# decide on number of dicrete times:
#halflife=13.*365.25

#breaks = 365.25*np.r_[1, np.arange(2, endpt+1, 0.5)]
breaks = 365.25*np.arange(1, endpt,0.5)
#breaks=-np.log(1-np.arange(0.0,0.96,0.05))*halflife/np.log(2) 
n_intervals=len(breaks)-1
timegap = breaks[1:] - breaks[:-1]



In [11]:
# intialize hyperparam:
hidden_layers_sizes = 4
n_epochs = 50000
learning_rate = 0.001
lr_decay = 0.001
momentum = 0.9
n_in = data_full.shape[1]-3 # number of features

# Current Set of Hyperparams:
learning_rate = 0.002119393184919567
hidden_layers_size = 2
patience = 90.24921875000001
l2 = 0.3291480859375
momentum = 0.7809667968749999
# Loss Train, Loss Valid, C-index Train, C-index Test:
# 0.274, 0.291, 0.821, 0.763

l2_final = l2

def nnet_pred_surv(y_pred, breaks, fu_time):
#Predicted survival probability from Nnet-survival model
#Inputs are Numpy arrays.
#y_pred: Rectangular array, each individual's conditional probability of surviving each time interval
#breaks: Break-points for time intervals used for Nnet-survival model, starting with 0
#fu_time: Follow-up time point at which predictions are needed
#
#Returns: predicted survival probability for each individual at specified follow-up time
  y_pred=np.cumprod(y_pred, axis=1)
  pred_surv = []
  for i in range(y_pred.shape[0]):
    pred_surv.append(np.interp(fu_time,breaks[1:],y_pred[i,:]))
  return np.array(pred_surv)

In [12]:
# Outerloop:
for fold in range(25):

    ## fold = 24
    print('')
    print('FOLD '+str(fold)+':')
    print('')

    trainingid = trainingid_all.iloc[:,fold]
    trainingid = trainingid[~np.isnan(trainingid)]
    eligible_id = data_full['ID'][data_full['ID'].isin(trainingid)]
    train_df = data_full.loc[data_full['ID'].isin(eligible_id),:]
    del train_df['ID']
    test_df = data_full.loc[~data_full['ID'].isin(eligible_id),:]
    del test_df['ID']


    # reformat the train and test set:
    y_train = nnet_survival.make_surv_array(train_df.time.values
                                            , train_df.event.values
                                            , breaks)
    y_test = nnet_survival.make_surv_array(test_df.time.values
                                           , test_df.event.values
                                           , breaks)
    featurespace_train_df = train_df.drop(['time','event'],axis =1)
    featurespace_test_df = test_df.drop(['time','event'],axis =1)

    featurespace_train = train_df.drop(['time','event'],axis =1).values
    featurespace_test = test_df.drop(['time','event'],axis =1).values



    ##########################
    # Build model:
    # l2_final=0.01

    from numpy.random import seed

    seed(1)
    import tensorflow as tf
    import keras
    tf.random.set_seed(2)

    model = Sequential()
    model.add(Dense(hidden_layers_sizes
                    , input_dim=featurespace_train.shape[1]
                    , bias_initializer='zeros'
                    , kernel_regularizer=regularizers.l2(l2_final)))
    model.add(Activation('relu'))
    model.add(Dense(n_intervals))
    model.add(Activation('sigmoid'))
    # import keras.backend.tensorflow_backend as kk

    model.compile(loss=nnet_survival.surv_likelihood(n_intervals), optimizer=optimizers.RMSprop())
    early_stopping = EarlyStopping(monitor='loss', patience=20)
    history=model.fit(featurespace_train, y_train, batch_size=4063, epochs=100000
                      , callbacks=[early_stopping]
                      , verbose=0)


    # print loss of train and valid data:
    # print(model.evaluate(featurespace_train,y_train,verbose=0))
    # model.evaluate(featurespace_test,y_test,verbose=0)

    #Discrimination performance
    # y_pred is conditional prob of survival within each time interval
    y_pred_train = model.predict(featurespace_train,verbose=0)
    # cumprod = cumulative product, the probability of surviving from time 0 up to the time interval of interest
    # index -1 because we are interested in -1 
    last_yr_surv_train=np.cumprod(y_pred_train[:,0:np.nonzero(breaks>365*(endpt-1))[0][0]], axis=1)[:,-1]
    print('Train C-index fold', str(fold+1),':')
    print(concordance_index(train_df.time, last_yr_surv_train, train_df.event)) 

    
    yr_endpt_surv_train = nnet_pred_surv(y_pred_train, breaks, 365.25*endpt)
    
    y_pred_test = model.predict_proba(featurespace_test ,verbose=0)
    yr_endpt_surv_test = nnet_pred_surv(y_pred_test, breaks, 365.25*endpt)

    c_index_train = concordance_index(train_df.time, yr_endpt_surv_train, train_df.event)
    c_index_test = concordance_index(test_df.time, yr_endpt_surv_test, test_df.event)

    
    
    y_pred=model.predict(featurespace_test,verbose=0)
    last_yr_surv=np.cumprod(y_pred[:,0:np.nonzero(breaks>365*(endpt-1))[0][0]], axis=1)[:,-1]
    print('Test C-index fold', str(fold+1),':')
    print(concordance_index(test_df.time,last_yr_surv, test_df.event))

    print('Train C-index2 fold', str(fold+1),':')
    print(c_index_train)
    print('Test C-index2 fold', str(fold+1),':')
    print(c_index_test)

    
    pred_surv = np.zeros((len(test_df.event), len(eval_times)))
    col=0
    for time in eval_times:
        pred_surv[:,col] = nnet_pred_surv(y_pred_test, breaks, time)
        col = col+1
    pred_surv = pd.DataFrame(data = pred_surv)
    # savedir = os.path.join(os.getcwd(),'python_files/csv_files/nnet_survival/all_features')
    savedir = os.path.join(work_dir,'csv_files/nnet_survival/'+str(n_features)+'_features/'+term_pred+'l2_0_1')
    try: 
        os.makedirs(savedir)
    except OSError:
        if not os.path.isdir(savedir):
            raise
    actual_fold = fold+1
    pred_surv.to_csv(savedir+'/pred_prob_surv_fold_'+str(actual_fold)+'.csv', index = None, header = True)






FOLD 0:

Train C-index fold 1 :
0.743689505933767


/home/idies/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


Test C-index fold 1 :
0.7424321639567104
Train C-index2 fold 1 :
0.7441334602901855
Test C-index2 fold 1 :
0.7414649448423695

FOLD 1:

Train C-index fold 2 :
0.7512133549939669
Test C-index fold 2 :
0.7233179576904724
Train C-index2 fold 2 :
0.7502106519086749
Test C-index2 fold 2 :
0.7227347436509199

FOLD 2:

Train C-index fold 3 :
0.7456846739734969
Test C-index fold 3 :
0.7380607814761215
Train C-index2 fold 3 :
0.7456855065658231
Test C-index2 fold 3 :
0.7380607814761215

FOLD 3:

Train C-index fold 4 :
0.7628152008627571
Test C-index fold 4 :
0.6752771414277062
Train C-index2 fold 4 :
0.7619249340527476
Test C-index2 fold 4 :
0.6777259920478185

FOLD 4:

Train C-index fold 5 :
0.7376248847634337
Test C-index fold 5 :
0.7880388199187104
Train C-index2 fold 5 :
0.7364637669562376
Test C-index2 fold 5 :
0.7830895567782785

FOLD 5:

Train C-index fold 6 :
0.750927319054976
Test C-index fold 6 :
0.7258799392502198
Train C-index2 fold 6 :
0.7505436008253308
Test C-index2 fold 6 :
0.72